In [ ]:
#make sure you install all these modules first, you can do that in cmd or jupyter notebook itself
#can use "pip install pandas" to install (might change for diff interfaces)

import pandas as pd
import numpy as np
import praw
import math
import datetime as dt
from newspaper import Article
import nltk
from __future__ import print_function
import time
import datetime as dt
import sib_api_v3_sdk
from sib_api_v3_sdk.rest import ApiException
from pprint import pprint

In [ ]:
#Loading users database for the newsletter

users= pd.read_excel(r"C:\Users\mssre\Desktop\Psytech\user survey mutter digest\users_60.xlsx") #change this path of file to your local user database
users['Email Address']= users['Email Address'].str.lower()
users.dropna(subset=['Email Address'],inplace=True)

#this step is to reduce manual subreddit entry, subreddits are case sensitive
users['fav']= ((users['Cat1']+","+users['Cat2']+","+users['Cat3'])).astype('str')
users['interests']= users['fav'].apply(lambda x: list(x.split(',')))
users.drop('fav',axis=1,inplace=True)
tag= users['interests'].sum()
tags= list(set(tag))
a=' '.join(tags)
a= a.replace(' ','+').strip()
a

In [ ]:
#just to display the dataframe

users

In [ ]:
    reddit = praw.Reddit(client_id='Sqc3X3Z6cfeHDbybfiiW9Q', 
                     client_secret='c83zLdpcAHiXCDg5V5unVL5BpyLBkQ', 
                     user_agent='scraper_101', 
                     username='lotta_true_crime ', 
                     password='renuka21') #do not change, all these details
    
    #Initiating the scraper
    no_subreddit = reddit.subreddit(a)
    new = no_subreddit.new(limit=30000) #limit can be changed
    dict =    { "title":[],
                "subreddit":[],
                "Upvotes":[],
                "Upvote_ratio":[],
                "id":[], 
                "url":[], 
                "comms_num": [], 
                "created": [], 
                "body":[],
                "NSFW(Not safe for work)":[] }
    
    #creating a dataframe of scraped data
    for submission in new:
        dict["title"].append(submission.title)
        dict['subreddit'].append(submission.subreddit)
        dict["Upvotes"].append(submission.score)
        dict["id"].append(submission.id)
        dict["url"].append(submission.url)
        dict['Upvote_ratio'].append(submission.upvote_ratio)
        dict["comms_num"].append(submission.num_comments)
        dict["created"].append(submission.created)
        dict["body"].append(submission.selftext)
        dict["NSFW(Not safe for work)"].append(submission.over_18)
    df = pd.DataFrame(dict)
    df.drop_duplicates(subset=['id'], inplace=True)

    #correcting datetime format
    def get_date(created):
        return dt.datetime.fromtimestamp(created)
    x=[]
    for i in df['created']:
        x.append(get_date(i))
    df['DateTime']=x
    df.drop(['created'],axis=1,inplace=True)
    
    #sorting posts
    df.sort_values(['Upvotes','Upvote_ratio','DateTime'],ascending=False,inplace=True)
    
    #dropping image and video content based on url(optimized version)
    match = ['img','jpg','gif','png','jpeg','imgur','gfy','reddit','redd.it','youtube','youtu.be','comments','comment']
    df['pic'] = df['url'].apply(lambda x: 1 if any(s in x for s in match) else 0)
    df_notpic= df[df['pic']==0]
    df_notpic= df[df['NSFW(Not safe for work)']==0]
       
    #Setting a time filter, set as 1 day max old
    timeline= dt.datetime.today()-dt.timedelta(1) 
    df_notpic= df_notpic[df_notpic['DateTime']>timeline]
    df_notpic['subreddit']= df_notpic['subreddit'].astype('str')
    
    #selects top 5 from each group and extracts text & summary
    df_mail= df_notpic.groupby('subreddit').nth(list(range(20)))
  
    #Creating another column for fetching summary of each article using Newspaper3k module
    b=[]
    for k in df_mail['url']:
        arc= Article(k)
        try:
            arc.download()
            arc.parse()
            arc.nlp()
            summ=arc.summary
        except:
            summ=np.nan
        b.append(summ)
    df_mail['Summary']=b
    df_mail['Summary']= df_mail['Summary'].astype('str')    

In [ ]:
#this to make sure summary is not blank, and just to format into a better dataframe for easy accesibility

df_mail['leng']= df_mail['Summary'].apply(lambda x: len(x))
df_mail= df_mail[df_mail['leng']>30]
df_mail.dropna(subset=['Summary'],inplace=True)
df_mail['r']=df_mail['url'].str.contains('reddit').astype('int')
df_mail['r1']=df_mail['url'].str.contains('redd.it').astype('int')
df_mail['red']= df_mail['r']+df_mail['r1']
df_mail= df_mail[df_mail['red']==0]
df_mail.reset_index(inplace=True)

In [ ]:
#optional to run. just to make sure each and every user have enough content (>10 rows)

for i in range(len(users['interests'])):
    r= users['interests'].iloc[i]
    df_mail1 = df_mail[df_mail['subreddit'].isin(r)]
    print(df_mail1.shape,i)

In [ ]:
#this function sorts, formats and sends the mail
#takes email and list of interests as input

def custom_mailer(email,r):
    df_mail1 = df_mail[df_mail['subreddit'].isin(r)]
    df_mails=df_mail1.groupby('subreddit').nth(list(range(10))).sample(10)
    df_mails.reset_index(inplace=True)
    
    df_mailer= pd.DataFrame()
    df_mailer['title']= df_mails['title']
    df_mailer['Link to the post']= df_mails['url']
    df_mailer['Summary']= df_mails['Summary']
    df_mailer['Stream']= df_mails['subreddit']
    df_mailer
    
    #html template for the mail
    html_content = '''<!DOCTYPE html>
        <html>
            <head>
                <title>Top 10 Picks Just For You</title>
                <style>
                .button {{border: none;
                 color: white;
                 padding: 8px 18px;
                 text-align: center;
                 text-decoration: none;
                 display: inline-block;
                 font-size: 14px;
                 margin: 6px 2px;
                 cursor: pointer;}}
                    @import url('https://fonts.googleapis.com/css2?family=Nunito&family=Nunito+Sans&display=swap');

                    /* SETTING TEXT CSS PROPERTIES */
                    .b1 {{ font-size: 16px; }}
                    .b2 {{ font-size: 14px; }}
                    .b3 {{ font-size: 14px; }}
                    .h1 {{ font-size: 40px; }}
                    .h2 {{ font-size: 22px; }}                    
                    .button1 {{background-color: #4CAF50;}}
                    .button2 {{background-color: #008CBA;}}
                    .fw-bold {{ font-weight: bold; }}
                    .fw-bolder {{ font-weight: bolder; }}
                    .text-start {{ text-align: start; }}
                    .text-center {{ text-align: center; }}
                    .text-decoration-none {{ text-decoration: none; }}
                    .nunito {{ font-family: 'Nunito', sans-serif; }}
                    .nunito-sans {{ font-family: 'Nunito Sans', sans-serif; }}

                    /* SETTING COLORS & BACKGROUND COLORS */
                    .text-dark {{ color: #291238!important; }}
                    .text-secondary {{ color: #666666; }}
                    .bg-white {{ background: #FFFFFF; }}
                    .bg-container {{ background-image: linear-gradient(#291238, #FFF); }}

                    .w-75 {{ width: 75%; }}
                    .wa-75 {{ width: 75%; }}
                    .p-3 {{ padding: 15px; }}
                    .pt-3 {{ padding-top: 17px; }}
                    .pt-5 {{ padding-top: 25px; }}
                    .mb-5 {{ margin-bottom: 25px; }}
                    .pt-2 {{ margin-top: 15px; }}
                    .pt-1 {{ margin-top: 7px; }}
                    .my-2 {{ margin-top: 15px; margin-bottom: 15px; }}

                    /* SETTING WIDTH, MARGIN & PROPERTIES */
                    .centered-box {{
                        display: block;
                        margin-left: auto;
                        margin-right: auto;
                    }}

                    @media (max-width: 600px) {{
                        .wa-75 {{ width: 95%; }}
                    }}
                </style>
            </head>
            <body class="text-center nunito-sans bg-container">

                <div class="bg-white w-75 pt-5 p-3 centered-box">

                    <h1 class="nunito fw-bolder pt-2">Mutter Buzz</h1>
                    <h2 class="nunito fw-bold pt-2">Your inbox just got more personalized!</h2>
                    <p class="b2">Hey {name}, here's your daily dose of internet, personally curated just for you! Drop your feedback and help us give you the best experience. Liked it? Feel free to forward Mutter Buzz among your friends & loved ones, cause sharing is caring :)</p>

                    
                        <div class="text-start centered-box wa-75 my-2">
                            <a href={link1} class="b1 fw-bolder text-dark text-decoration-none">{title1}</a>
                            <div class="b3 text-secondary">Category: {category1} | {readTime1} minute read</div>
                            <div class="b2">{description1}</div>                            
                            
                        </div>

                        <div class="text-start centered-box wa-75 my-2">
                            <a href={link2} class="b1 fw-bolder text-dark text-decoration-none">{title2}</a>
                            <div class="b3 text-secondary">Category: {category2} | {readTime2} minute read</div>
                            <div class="b2">{description2}</div>
                        </div>

                        <div class="text-start centered-box wa-75 my-2">
                            <a href={link3} class="b1 fw-bolder text-dark text-decoration-none">{title3}</a>
                            <div class="b3 text-secondary">Category: {category3} | {readTime3} minute read</div>
                            <div class="b2">{description3}</div>
                        </div>

                        <div class="text-start centered-box wa-75 my-2">
                            <a href={link4} class="b1 fw-bolder text-dark text-decoration-none">{title4}</a>
                            <div class="b3 text-secondary">Category: {category4} | {readTime4} minute read</div>
                            <div class="b2">{description4}</div>
                        </div>

                        <div class="text-start centered-box wa-75 my-2">
                            <a href={link5} class="b1 fw-bolder text-dark text-decoration-none">{title5}</a>
                            <div class="b3 text-secondary">Category: {category5} | {readTime5} minute read</div>
                            <div class="b2">{description5}</div>
                        </div>

                        <div class="text-start centered-box wa-75 my-2">
                            <a href={link6} class="b1 fw-bolder text-dark text-decoration-none">{title6}</a>
                            <div class="b3 text-secondary">Category: {category6} | {readTime6} minute read</div>
                            <div class="b2">{description6}</div>
                        </div>

                        <div class="text-start centered-box wa-75 my-2">
                            <a href={link7} class="b1 fw-bolder text-dark text-decoration-none">{title7}</a>
                            <div class="b3 text-secondary">Category: {category7} | {readTime7} minute read</div>
                            <div class="b2">{description7}</div>
                        </div>

                        <div class="text-start centered-box wa-75 my-2">
                            <a href={link8} class="b1 fw-bolder text-dark text-decoration-none">{title8}</a>
                            <div class="b3 text-secondary">Category: {category8} | {readTime8} minute read</div>
                            <div class="b2">{description8}</div>
                        </div>

                        <div class="text-start centered-box wa-75 my-2">
                            <a href={link9} class="b1 fw-bolder text-dark text-decoration-none">{title9}</a>
                            <div class="b3 text-secondary">Category: {category9} | {readTime9} minute read</div>
                            <div class="b2">{description9}</div>
                        </div>

                        <div class="text-start centered-box wa-75 my-2">
                            <a href={link10} class="b1 fw-bolder text-dark text-decoration-none">{title10}</a>
                            <div class="b3 text-secondary">Category: {category10} | {readTime10} minute read</div>
                            <div class="b2">{description10}</div>
                       </div>
                        <a href="https://docs.google.com/forms/d/e/1FAIpQLScRWJGR9-gTR7L3XyBdHAFk89g7zH_AbxHc0Ek-3RR5vWJ1aw/viewform">
                            <button class="button button1">Subscribe</button></a>
                        </div>
                        <a href="https://forms.gle/Ao6VitoA4WDvNU5bA">
                            <button class="button button2">Rate us, every feedback matters</button></a>                                             
                    
                    <div class="pt-3 b2">Sent by <a class="text-dark fw-bold" href="https://psytech.ai">PsyTech</a>, South Extenstion II, New Delhi - 110049</div>
                </div>
            </body>
        </html>
    '''.format(
        
        name = name,
        link1=df_mailer.iloc[0]["Link to the post"],
        title1=df_mailer.iloc[0]["title"],
        description1=df_mailer.iloc[0]["Summary"],
        readTime1=math.ceil(4 * len(df_mailer.iloc[0]["Summary"].split(" "))/200),
        category1=df_mailer.iloc[0]["Stream"],
       

        link2=df_mailer.iloc[1]["Link to the post"],
        title2=df_mailer.iloc[1]["title"],
        description2=df_mailer.iloc[1]["Summary"],
        readTime2=math.ceil(4 * len(df_mailer.iloc[1]["Summary"].split(" "))/200),
        category2=df_mailer.iloc[1]["Stream"],
        

        link3=df_mailer.iloc[2]["Link to the post"],
        title3=df_mailer.iloc[2]["title"],
        description3=df_mailer.iloc[2]["Summary"],
        readTime3=math.ceil(4 * len(df_mailer.iloc[2]["Summary"].split(" "))/200),
        category3=df_mailer.iloc[2]["Stream"],

        link4=df_mailer.iloc[3]["Link to the post"],
        title4=df_mailer.iloc[3]["title"],
        description4=df_mailer.iloc[3]["Summary"],
        readTime4=math.ceil(4 * len(df_mailer.iloc[3]["Summary"].split(" "))/200),
        category4=df_mailer.iloc[3]["Stream"],

        link5=df_mailer.iloc[4]["Link to the post"],
        title5=df_mailer.iloc[4]["title"],
        description5=df_mailer.iloc[4]["Summary"],
        readTime5=math.ceil(4 * len(df_mailer.iloc[4]["Summary"].split(" "))/200),
        category5=df_mailer.iloc[4]["Stream"],

        link6=df_mailer.iloc[5]["Link to the post"],
        title6=df_mailer.iloc[5]["title"],
        description6=df_mailer.iloc[5]["Summary"],
        readTime6=math.ceil(4 * len(df_mailer.iloc[5]["Summary"].split(" "))/200),
        category6=df_mailer.iloc[5]["Stream"],

        link7=df_mailer.iloc[6]["Link to the post"],
        title7=df_mailer.iloc[6]["title"],
        description7=df_mailer.iloc[6]["Summary"],
        readTime7=math.ceil(4 * len(df_mailer.iloc[6]["Summary"].split(" "))/200),
        category7=df_mailer.iloc[6]["Stream"],

        link8=df_mailer.iloc[7]["Link to the post"],
        title8=df_mailer.iloc[7]["title"],
        description8=df_mailer.iloc[7]["Summary"],
        readTime8=math.ceil(4 * len(df_mailer.iloc[7]["Summary"].split(" "))/200),
        category8=df_mailer.iloc[7]["Stream"],

        link9=df_mailer.iloc[8]["Link to the post"],
        title9=df_mailer.iloc[8]["title"],
        description9=df_mailer.iloc[8]["Summary"],
        readTime9=math.ceil(4 * len(df_mailer.iloc[8]["Summary"].split(" "))/200),
        category9=df_mailer.iloc[8]["Stream"],

        link10=df_mailer.iloc[9]["Link to the post"],
        title10=df_mailer.iloc[9]["title"],
        description10=df_mailer.iloc[9]["Summary"],
        readTime10=math.ceil(4 * len(df_mailer.iloc[9]["Summary"].split(" "))/200),
        category10=df_mailer.iloc[9]["Stream"])        
    
    #Initiating send in blue API
    # to send the mail
    configuration = sib_api_v3_sdk.Configuration()
    configuration.api_key['api-key'] ='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'  #do not change

    api_instance = sib_api_v3_sdk.TransactionalEmailsApi(sib_api_v3_sdk.ApiClient(configuration))
    subject = "Mutter Buzz :: Created by Humans, Curated by AI"
    sender = {"name":"Renuka from PsyTech","email":"renuka@psytech.ai"} #can be changed, limited to send in blue users only
    replyTo = {"name":"Renuka Meruva","email":"renuka@psytech.ai"} #can be customised
    to = [{"email":email}]
    send_smtp_email = sib_api_v3_sdk.SendSmtpEmail(to=to, html_content=html_content, sender=sender, subject=subject,reply_to=replyTo)
    try:
        api_response = api_instance.send_transac_email(send_smtp_email)
        return api_response
    except ApiException as e:
        return ("Exception when calling EmailCampaignsApi->create_email_campaign: %s\n" % e)
    

In [ ]:
#calling the functions in for loop

for i in range(len(users['Email Address'])):
    r= list(users['interests'].iloc[i])
    name= users['First Name'].iloc[i]
    email= users['Email Address'].iloc[i]
    print(custom_mailer(email,r))

## This is to send report of transactional mail history

### Not required for the run 


configuration = sib_api_v3_sdk.Configuration()
configuration.api_key['api-key'] = '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'

api_instance0 = sib_api_v3_sdk.TransactionalEmailsApi(sib_api_v3_sdk.ApiClient(configuration))
start_date = (dt.datetime.now()-dt.timedelta(1)).date()
end_date =(dt.datetime.now()-dt.timedelta(1)).date()
api_response0 = api_instance0.get_aggregated_smtp_report(start_date=start_date, end_date=end_date)

api_instance1 = sib_api_v3_sdk.EmailCampaignsApi(sib_api_v3_sdk.ApiClient(configuration))
tag = 'mutter_buzz_report'
subject = "Mutter Buzz aggregate Report of "+ str((dt.datetime.now()-dt.timedelta(1)).date())
sender = {"email":"renuka@psytech.ai"}
name= "Mutter Buzz aggregate Report- "+ str((dt.datetime.now()-dt.timedelta(1)).date())
recipients = {"listIds": [22]}
scheduled_at = dt.datetime.now()+dt.timedelta(hours=6)

html_content= '''<!DOCTYPE html>
        <html>
        <body><h3>{report}</h3></body></html>'''.format(report=api_response0)

email_campaigns = sib_api_v3_sdk.CreateEmailCampaign(tag=tag, sender=sender, name=name, html_content=html_content, scheduled_at=scheduled_at, subject=subject, recipients=recipients)

try:
    api_response = api_instance1.create_email_campaign(email_campaigns)
    pprint(api_response)
except ApiException as e:
    print("Exception when calling EmailCampaignsApi->create_email_campaign: %s\n" % e)